In [1]:
"""Sets the notebook to the project root"""

import sys, os
PROJECT_NAME = "sitecards"
current_directory = os.getcwd().split("/")
while current_directory[-1] != PROJECT_NAME:
    current_directory = current_directory[:-1]
if len(current_directory) > 1:
    project_root = "/".join(current_directory)
    os.chdir(project_root)
else:
    raise Exception("ERROR: Project root not found")

from dotenv import load_dotenv
load_dotenv(dotenv_path="./env/env.env")

True

In [34]:
#!/usr/bin/env python3

import argparse
from google.cloud import storage
from google.auth.exceptions import DefaultCredentialsError

import pylandmarks.mongo.database as database
from bson import ObjectId

def format_size(size_bytes):
    """Converts a size in bytes to a human-readable string."""
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = 0
    while size_bytes >= 1024 and i < len(size_name) - 1:
        size_bytes /= 1024.0
        i += 1
    # Format to one decimal place if it's not an integer, otherwise no decimal places
    if size_bytes == int(size_bytes):
        return f"{int(size_bytes)}{size_name[i]}"
    else:
        return f"{size_bytes:.1f}{size_name[i]}"

if __name__ == "__main__":
    gcs_url = "gs://media-owner-sitecards/sitecards/JCD_X_ALL_AIRPORT_page_141.png"
    

    print("------------------------------------")
    
    """Initialize database connections."""
    dbs = database.initialize({
        "stg": os.environ["MONGO_STAGE_URI"], 
        "prd": os.environ["MONGO_PROD_URI"]
    })

    sitecards = list(dbs["stg"]["poinetwork"]["sitecards"].find())

    for sitecard in sitecards:
        gcs_url = sitecard["metadata"]["gcs_url"]
        panel_id = sitecard["processed_data"]["panel_id"]
        # Parse the GCS URL to extract bucket name and object path
        if gcs_url.startswith("gs://"):
            # Remove the gs:// prefix
            path_without_prefix = gcs_url[5:]
        # Split into bucket name and object path
        parts = path_without_prefix.split("/", 1)
        bucket_name = parts[0]
        object_path = parts[1] if len(parts) > 1 else ""

        # Initialize the GCS client
        storage_client = storage.Client()

        # Get the bucket
        bucket = storage_client.bucket(bucket_name)
        # Get the specific blob
        blob = bucket.blob(object_path)
        blob.reload()
        size = format_size(blob.size)
        print(f"File: {blob.name}, size: {size} ({blob.size} bytes), panel_id: {panel_id}")
        
    print("------------------------------------")


------------------------------------
File: sitecards/Go_Media-_ALL_DIGI_page_1.png, size: 4.6MB (4783361 bytes), panel_id: Not found
File: sitecards/Go_Media-_ALL_DIGI_page_10.png, size: 4.6MB (4826664 bytes), panel_id: AUCKLED08
File: sitecards/Go_Media-_ALL_DIGI_page_100.png, size: 2.8MB (2897129 bytes), panel_id: HAMILED08
File: sitecards/Go_Media-_ALL_DIGI_page_101.png, size: 3.6MB (3824243 bytes), panel_id: HAMILED09
File: sitecards/Go_Media-_ALL_DIGI_page_102.png, size: 3.8MB (3979449 bytes), panel_id: TAURLED01
File: sitecards/Go_Media-_ALL_DIGI_page_103.png, size: 3.3MB (3425880 bytes), panel_id: TAURLED02
File: sitecards/Go_Media-_ALL_DIGI_page_104.png, size: 2.8MB (2978549 bytes), panel_id: TAURLED03
File: sitecards/Go_Media-_ALL_DIGI_page_105.png, size: 2.6MB (2736718 bytes), panel_id: TAURLED05
File: sitecards/Go_Media-_ALL_DIGI_page_106.png, size: 3.4MB (3529579 bytes), panel_id: TAURLED08
File: sitecards/Go_Media-_ALL_DIGI_page_107.png, size: 5.5MB (5762400 bytes), panel_